# 🚚 Freight Pricing Model Export - Final Production

This notebook trains and exports all pricing models for the Freight Pricing Tool.

## Model Architecture

**Pricing Cascade (in order of priority):**
1. **Recency Model** - Median of loads within last 90 days (if available)
2. **Index + Shrinkage Model** - For lanes WITH historical data (uses market index + shrinkage estimation)
3. **Province Blend 0.7 + LAD Model** - For lanes with NO historical data (70% province-level CPK + 30% city decomposition, with LAD calibration)

## Key Improvements in This Version
- **Province-based regions** (13 provinces) instead of 5 custom freight regions
- **LAD (L1) calibration** applied to Blend model predictions to minimize MAE
- **Comprehensive validation** for each model component

## Output Files (to `final_models/`)
| File | Description |
|------|-------------|
| `reference_data.csv` | Historical trip data for KNN lookups |
| `config.pkl` | Configuration and distance lookups |
| `rare_lane_models.pkl` | Index + Shrinkage model (lanes WITH history) |
| `new_lane_model_blend.pkl` | Province Blend 0.7 + LAD model (NEW lanes) |
| `distance_matrix.pkl` | City-to-city distance lookup |
| `city_normalization_with_regions.csv` | City normalization and province mapping |
| `carrier_model.json` | CatBoost model (optional) |

---
# ⚙️ GLOBAL CONFIGURATION

Edit these settings before running the notebook.

In [141]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║                         FILE PATHS                                   ║
# ╚══════════════════════════════════════════════════════════════════════╝

# Main trip data file (required)
TRIP_DATA_FILE = 'Pfull5.csv'

# Supporting files
BACKHAUL_FILE = 'Backhaul Prob.csv'
IMBALANCE_FILE = 'route imbalance.csv'
DISTANCE_FILE = '/Users/maherelghor/Documents/All Lane Pricing - Distances.csv'

# City regions file - use region_test.csv which has both 'region' (5 regions) and 'province' (13 provinces)
CITY_REGIONS_FILE = '/Users/maherelghor/Documents/city_normalization_with_regions.csv'

# Output directory
EXPORT_DIR = 'final_models'

# ╔══════════════════════════════════════════════════════════════════════╗
# ║                      DATA FILTERS                                    ║
# ╚══════════════════════════════════════════════════════════════════════╝

ENTITY_MAPPING = 'Domestic'
VEHICLE_TYPES = ['تريلا فرش', 'تريلا ستائر']  # Flatbed and Curtain trailers
MIN_CARRIER_PRICE = 50
MAX_CARRIER_PRICE = 100000
MIN_DISTANCE = 50  # km
MIN_CPK = 0.5  # SAR/km
MAX_CPK = 5.0  # SAR/km

print("✅ Global configuration loaded")
print(f"📁 Export directory: {EXPORT_DIR}/")

✅ Global configuration loaded
📁 Export directory: final_models/


---
# 📚 IMPORTS & SETUP

In [142]:
import pandas as pd
import numpy as np
import pickle
import os
import shutil
import warnings
from scipy.optimize import minimize
warnings.filterwarnings('ignore')

# Create export directory
os.makedirs(EXPORT_DIR, exist_ok=True)

print("✅ Libraries loaded")
print(f"📁 Export directory: {os.path.abspath(EXPORT_DIR)}")

✅ Libraries loaded
📁 Export directory: /Users/maherelghor/Downloads/final_models


---
# 📁 LOAD & PREPARE DATA

In [143]:
# Load main trip data
print("Loading trip data...")
df_raw = pd.read_csv(TRIP_DATA_FILE)
print(f"   Raw: {len(df_raw):,} rows")

# Clean numeric columns
for col in ['distance', 'total_carrier_price', 'total_shipper_price', 'weight', 'cost']:
    if col in df_raw.columns:
        df_raw[col] = pd.to_numeric(
            df_raw[col].astype(str).str.replace(',', ''), 
            errors='coerce'
        )

# Handle different column names for cost
if 'cost' in df_raw.columns and 'total_carrier_price' not in df_raw.columns:
    df_raw['total_carrier_price'] = df_raw['cost']

# Parse dates
df_raw['pickup_date'] = pd.to_datetime(df_raw['pickup_date'], format='mixed', errors='coerce')

print(f"   Date range: {df_raw['pickup_date'].min()} to {df_raw['pickup_date'].max()}")

Loading trip data...
   Raw: 95,634 rows
   Date range: 2020-03-06 13:00:00 to 2026-01-21 08:00:00


In [144]:
# Load supporting files
print("\nLoading supporting files...")

# Backhaul data
try:
    backhaul_df = pd.read_csv(BACKHAUL_FILE)
    print(f"   ✅ Backhaul: {len(backhaul_df)} cities")
except FileNotFoundError:
    backhaul_df = pd.DataFrame(columns=['destination_city', 'backhaul_probability_pct'])
    print("   ⚠️ Backhaul file not found - skipping")

# Imbalance data
try:
    imbalance_df = pd.read_csv(IMBALANCE_FILE)
    imbalance_df['load_count'] = pd.to_numeric(
        imbalance_df['load_count'].astype(str).str.replace(',', ''), 
        errors='coerce'
    )
    imbalance_df['lane'] = imbalance_df['pickup_city'] + ' → ' + imbalance_df['destination_city']
    print(f"   ✅ Imbalance: {len(imbalance_df)} lanes")
except FileNotFoundError:
    imbalance_df = pd.DataFrame(columns=['lane', 'load_count', 'load_imbalance'])
    print("   ⚠️ Imbalance file not found - skipping")

# City regions/provinces - region_test.csv has both 'region' (5 regions) and 'province' (13 provinces)
try:
    city_regions_df = pd.read_csv(CITY_REGIONS_FILE)
    
    # Create PROVINCE mapping (13 provinces) - for Blend model
    # Uses the 'province' column from region_test.csv
    city_to_province = dict(zip(city_regions_df['variant'], city_regions_df['province']))
    city_to_province.update(dict(zip(city_regions_df['canonical'], city_regions_df['province'])))
    # Remove None/NaN values
    city_to_province = {k: v for k, v in city_to_province.items() if pd.notna(v) and v != ''}
    
    # Create REGION mapping (5 regions) - for Index+Shrinkage fallback
    # Uses the 'region' column from region_test.csv
    city_to_region = dict(zip(city_regions_df['variant'], city_regions_df['region']))
    city_to_region.update(dict(zip(city_regions_df['canonical'], city_regions_df['region'])))
    # Remove None/NaN values
    city_to_region = {k: v for k, v in city_to_region.items() if pd.notna(v) and v != ''}
    
    print(f"   ✅ City-to-province mappings (13 provinces): {len(city_to_province)} cities")
    print(f"   ✅ Unique provinces: {len(set(city_to_province.values()))}")
    print(f"   ✅ City-to-region mappings (5 regions): {len(city_to_region)} cities")
    print(f"   ✅ Unique regions: {len(set(city_to_region.values()))}")
    
    # Print the unique provinces and regions for verification
    print(f"   📋 Provinces: {sorted(set(city_to_province.values()))}")
    print(f"   📋 Regions: {sorted(set(city_to_region.values()))}")
    
except FileNotFoundError:
    city_to_province = {}
    city_to_region = {}
    print("   ⚠️ City regions file not found")

# Hardcoded distances
try:
    dist_df = pd.read_csv(DISTANCE_FILE)
    if len(dist_df.columns) >= 3:
        dist_df.columns = ['pickup_city', 'destination_city', 'distance_km'] + list(dist_df.columns[3:])
    dist_df['distance_km'] = pd.to_numeric(dist_df['distance_km'], errors='coerce')
    hardcoded_distances = {
        (row['pickup_city'], row['destination_city']): row['distance_km']
        for _, row in dist_df.iterrows()
        if pd.notna(row['distance_km']) and row['distance_km'] > 0
    }
    print(f"   ✅ Distances: {len(hardcoded_distances):,} city pairs")
except FileNotFoundError:
    hardcoded_distances = {}
    print("   ⚠️ Distance file not found")


Loading supporting files...
   ✅ Backhaul: 24 cities
   ✅ Imbalance: 289 lanes
   ✅ City-to-province mappings (13 provinces): 527 cities
   ✅ Unique provinces: 13
   ✅ City-to-region mappings (5 regions): 528 cities
   ✅ Unique regions: 5
   📋 Provinces: ['Al Bahah Province', 'Al Jouf Province', 'Asir Province', 'Eastern Province', 'Hail Province', 'Jazan Province', 'Madinah Province', 'Makkah Province', 'Najran Province', 'Northern Borders Province', 'Qassim Province', 'Riyadh Province', 'Tabuk Province']
   📋 Regions: ['Central', 'Eastern', 'Northern', 'Southern', 'Western']
   ✅ Distances: 4,492 city pairs


In [97]:
# Apply filters and clean data
print("Filtering data...")

df = df_raw[
    (df_raw['entity_mapping'] == ENTITY_MAPPING) &
    (df_raw['vehicle_type'].isin(VEHICLE_TYPES)) &
    (df_raw['total_carrier_price'] >= MIN_CARRIER_PRICE) &
    (df_raw['total_carrier_price'] <= MAX_CARRIER_PRICE) &
    (df_raw['distance'] >= MIN_DISTANCE) &
    (df_raw['pickup_date'].notna())
].copy()

print(f"   After entity/vehicle filter: {len(df):,}")

# Calculate CPK and filter outliers
df['cost_per_km'] = df['total_carrier_price'] / df['distance']
df = df[
    (df['cost_per_km'] >= MIN_CPK) & 
    (df['cost_per_km'] <= MAX_CPK)
].copy()

print(f"   After CPK filter: {len(df):,}")

# Create derived features
df['lane'] = df['pickup_city'] + ' → ' + df['destination_city']
df['date'] = df['pickup_date'].dt.date

# Calculate days_ago from most recent date
most_recent = df['pickup_date'].max()
df['days_ago'] = (most_recent - df['pickup_date']).dt.days

# Fill missing values
for col in ['commodity', 'vehicle_type', 'pickup_city', 'destination_city']:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

if 'weight' in df.columns:
    df['weight'] = df['weight'].fillna(df['weight'].median())
else:
    df['weight'] = 25.0

if 'container' in df.columns:
    df['container'] = df['container'].fillna(0).astype(int)
else:
    df['container'] = 0

if 'leg_name' in df.columns:
    df['is_multistop'] = (
        (df['leg_name'] == 'Trip') |
        ((df['pickup_city'] == df['destination_city']) & (df['distance'] > 100))
    ).astype(int)
else:
    df['is_multistop'] = 0

# Map provinces
df['pickup_province'] = df['pickup_city'].map(city_to_province)
df['dest_province'] = df['destination_city'].map(city_to_province)

print(f"\n✅ Clean data: {len(df):,} trips, {df['lane'].nunique()} unique lanes")
print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   Province coverage: {df['pickup_province'].notna().mean()*100:.1f}% pickup, {df['dest_province'].notna().mean()*100:.1f}% dest")

Filtering data...
   After entity/vehicle filter: 84,052
   After CPK filter: 80,524

✅ Clean data: 80,524 trips, 299 unique lanes
   Date range: 2020-03-06 to 2026-01-20
   Province coverage: 100.0% pickup, 99.2% dest


In [98]:
# Merge supporting data
print("Merging supporting data...")

# Backhaul
if 'destination_city' in backhaul_df.columns and 'backhaul_probability_pct' in backhaul_df.columns:
    backhaul_merge = backhaul_df[['destination_city', 'backhaul_probability_pct']].copy()
    backhaul_merge.columns = ['destination_city', 'dest_backhaul_prob']
    df = df.merge(backhaul_merge, on='destination_city', how='left')
    df['dest_backhaul_prob'] = df['dest_backhaul_prob'].fillna(20.0)
    print(f"   ✅ Backhaul data merged")
else:
    df['dest_backhaul_prob'] = 20.0

# Imbalance
if 'lane' in imbalance_df.columns:
    imb_merge = imbalance_df[['lane', 'load_count', 'load_imbalance']].copy()
    imb_merge.columns = ['lane', 'lane_load_count', 'lane_imbalance']
    df = df.merge(imb_merge, on='lane', how='left')
    df['lane_load_count'] = df['lane_load_count'].fillna(0)
    df['lane_imbalance'] = df['lane_imbalance'].fillna(1.0)
    print(f"   ✅ Imbalance data merged")
else:
    df['lane_load_count'] = 0
    df['lane_imbalance'] = 1.0

print(f"\n✅ Final dataset: {len(df):,} rows")

Merging supporting data...
   ✅ Backhaul data merged
   ✅ Imbalance data merged

✅ Final dataset: 84,948 rows


---
# 📊 MODEL 1: MARKET INDEX

Builds a market-wide CPK index from benchmark (high-volume) lanes.
Used by both Index+Shrinkage and Blend models.

In [138]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║                   MARKET INDEX PARAMETERS                            ║
# ╚══════════════════════════════════════════════════════════════════════╝

N_BENCHMARK_LANES = 5       # Number of top lanes to use for index
INDEX_SMOOTHING_DAYS = 7    # Rolling average window for smoothing
INDEX_LOOKBACK_DAYS = 365   # 1 Year Lookback

print("Market Index Parameters:")
print(f"   Benchmark lanes: {N_BENCHMARK_LANES}")
print(f"   Smoothing window: {INDEX_SMOOTHING_DAYS} days")

Market Index Parameters:
   Benchmark lanes: 5
   Smoothing window: 7 days


In [127]:
# Identify top benchmark lanes
lane_counts = df['lane'].value_counts()
benchmark_lanes = lane_counts.head(N_BENCHMARK_LANES).index.tolist()

print("📊 Benchmark Lanes (by volume):")
for i, lane in enumerate(benchmark_lanes, 1):
    count = lane_counts[lane]
    print(f"   {i}. {lane}: {count:,} trips")

# Build daily market index
# Filter for benchmark lanes AND recency (last 365 days)
benchmark_df = df[
    (df['lane'].isin(benchmark_lanes)) & 
    (df['days_ago'] <= INDEX_LOOKBACK_DAYS)
]
daily_index = benchmark_df.groupby('date')['cost_per_km'].median().reset_index()
daily_index.columns = ['date', 'index_value']
daily_index = daily_index.sort_values('date')

# Smooth with rolling average
daily_index['index_smoothed'] = daily_index['index_value'].rolling(
    INDEX_SMOOTHING_DAYS, min_periods=1, center=True
).mean()

current_index = float(daily_index['index_smoothed'].iloc[-1])

print(f"\n✅ Market Index Built")
print(f"   Current index: {current_index:.4f} SAR/km")
print(f"   Index range: {daily_index['index_smoothed'].min():.3f} - {daily_index['index_smoothed'].max():.3f}")

📊 Benchmark Lanes (by volume):
   1. جدة → جدة: 22,195 trips
   2. الجبيل → جدة: 9,939 trips
   3. الدمام → جدة: 5,643 trips
   4. جدة → الرياض: 5,227 trips
   5. الرياض → جدة: 5,169 trips

✅ Market Index Built
   Current index: 1.7109 SAR/km
   Index range: 1.155 - 2.537


In [128]:
# Calculate lane multipliers (lane CPK relative to index)
df_indexed = df.merge(daily_index[['date', 'index_smoothed']], on='date', how='left')
df_indexed['index_smoothed'] = df_indexed['index_smoothed'].ffill().bfill()
df_indexed['multiplier'] = df_indexed['cost_per_km'] / df_indexed['index_smoothed']

lane_multipliers = df_indexed.groupby('lane')['multiplier'].median().to_dict()

print(f"✅ Lane multipliers: {len(lane_multipliers)}")
print("\nSample multipliers:")
for lane in benchmark_lanes[:3]:
    mult = lane_multipliers.get(lane, 1.0)
    print(f"   {lane}: {mult:.3f}x")

✅ Lane multipliers: 299

Sample multipliers:
   جدة → جدة: 0.961x
   الجبيل → جدة: 1.023x
   الدمام → جدة: 1.071x


---
# 🎯 MODEL 2: INDEX + SHRINKAGE (rare_lane_models.pkl)

For lanes **WITH** some historical data. Combines:
- **Index component**: Lane multiplier × current market index
- **Shrinkage component**: Bayesian estimate that pulls sparse lanes toward city priors

Final prediction = average of Index and Shrinkage components

In [102]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║               INDEX + SHRINKAGE PARAMETERS                           ║
# ╚══════════════════════════════════════════════════════════════════════╝

SHRINKAGE_K = 10    # Prior strength: higher = more shrinkage to city average
                    # At k=10, a lane needs ~10 trips to rely 50% on its own mean

print("Index + Shrinkage Parameters:")
print(f"   Prior strength (k): {SHRINKAGE_K}")
print(f"   Interpretation: Lane with n trips gets weight = n/(n+{SHRINKAGE_K})")

Index + Shrinkage Parameters:
   Prior strength (k): 10
   Interpretation: Lane with n trips gets weight = n/(n+10)


In [110]:
# Build Shrinkage Model components
print("Building Shrinkage Model...")

# Global mean CPK
global_mean = float(df['cost_per_km'].mean())

# City priors (average CPK for each city)
pickup_priors = df.groupby('pickup_city')['cost_per_km'].mean().to_dict()
dest_priors = df.groupby('destination_city')['cost_per_km'].mean().to_dict()

# Lane statistics for shrinkage
lane_stats = df.groupby('lane').agg(
    lane_mean=('cost_per_km', 'mean'),
    lane_n=('cost_per_km', 'count'),
    pickup_city=('pickup_city', 'first'),
    destination_city=('destination_city', 'first')
).to_dict('index')

print(f"\n✅ Shrinkage Model Built")
print(f"   Global mean: {global_mean:.3f} SAR/km")
print(f"   Pickup priors: {len(pickup_priors)} cities")
print(f"   Destination priors: {len(dest_priors)} cities")
print(f"   Lane stats: {len(lane_stats)} lanes")

Building Shrinkage Model...

✅ Shrinkage Model Built
   Global mean: 1.863 SAR/km
   Pickup priors: 35 cities
   Destination priors: 62 cities
   Lane stats: 299 lanes


In [111]:
# Build regional fallback (for Index+Shrinkage model, uses 5 regions)
print("Building 5-Region Model (for Index+Shrinkage fallback)...")

# city_to_region already contains the 5 macro-regions (Eastern, Western, Central, Northern, Southern)
# directly loaded from the 'region' column of region_test.csv
city_to_region_5 = city_to_region.copy()  # Already has 5 regions from earlier cell

# Map regions to dataframe
df['pickup_region_5'] = df['pickup_city'].map(city_to_region_5)
df['dest_region_5'] = df['destination_city'].map(city_to_region_5)

# Build regional CPK matrix (5 regions)
df_regional = df[df['pickup_region_5'].notna() & df['dest_region_5'].notna()]
regional_cpk_5 = df_regional.groupby(
    ['pickup_region_5', 'dest_region_5']
)['cost_per_km'].median().to_dict()

print(f"\n✅ 5-Region Model Built")
print(f"   Regional corridors: {len(regional_cpk_5)}")
print(f"   City-to-region mappings: {len(city_to_region_5)}")
print(f"   Region coverage: {df['pickup_region_5'].notna().mean()*100:.1f}% pickup, {df['dest_region_5'].notna().mean()*100:.1f}% dest")

Building 5-Region Model (for Index+Shrinkage fallback)...

✅ 5-Region Model Built
   Regional corridors: 24
   City-to-region mappings: 528
   Region coverage: 99.8% pickup, 94.3% dest


In [112]:
# Build distance lookups
historical_distances = df.groupby(
    ['pickup_city', 'destination_city']
)['distance'].median().to_dict()

print(f"📊 Distance Coverage:")
print(f"   Historical: {len(historical_distances)} pairs")
print(f"   Hardcoded:  {len(hardcoded_distances)} pairs")

historical_keys = set(historical_distances.keys())
hardcoded_keys = set(hardcoded_distances.keys())
print(f"   Overlap: {len(historical_keys & hardcoded_keys)} pairs")
print(f"   Total unique: {len(historical_keys | hardcoded_keys)} pairs")

📊 Distance Coverage:
   Historical: 312 pairs
   Hardcoded:  4492 pairs
   Overlap: 3 pairs
   Total unique: 4801 pairs


In [113]:
# Package rare_lane_models.pkl
rare_lane_artifacts = {
    # Distance lookups
    'historical_distances': historical_distances,
    'hardcoded_distances': hardcoded_distances,
    
    # Index model
    'current_index': current_index,
    'lane_multipliers': lane_multipliers,
    'benchmark_lanes': benchmark_lanes,
    
    # Shrinkage model
    'global_mean': global_mean,
    'k_prior_strength': SHRINKAGE_K,
    'pickup_priors': pickup_priors,
    'dest_priors': dest_priors,
    'lane_stats': lane_stats,
    
    # Regional fallback (5 regions)
    'city_to_region': city_to_region_5,
    'regional_cpk': regional_cpk_5,
    
    # Metadata
    'model_date': str(df['date'].max()),
    'training_trips': len(df),
    'training_lanes': df['lane'].nunique()
}

with open(f'{EXPORT_DIR}/rare_lane_models.pkl', 'wb') as f:
    pickle.dump(rare_lane_artifacts, f)

print(f"\n✅ Saved: {EXPORT_DIR}/rare_lane_models.pkl")


✅ Saved: final_models/rare_lane_models.pkl


### 🧪 VALIDATION: Index + Shrinkage Model

In [114]:
# Define predictor class for validation
class RareLanePredictor:
    def __init__(self, artifacts):
        self.current_index = artifacts['current_index']
        self.lane_multipliers = artifacts['lane_multipliers']
        self.global_mean = artifacts['global_mean']
        self.k = artifacts['k_prior_strength']
        self.pickup_priors = artifacts['pickup_priors']
        self.dest_priors = artifacts['dest_priors']
        self.lane_stats = artifacts['lane_stats']
        self.historical_distances = artifacts['historical_distances']
        self.hardcoded_distances = artifacts['hardcoded_distances']
    
    def get_distance(self, pickup, dest):
        key = (pickup, dest)
        if key in self.historical_distances:
            return self.historical_distances[key], 'historical'
        if key in self.hardcoded_distances:
            return self.hardcoded_distances[key], 'hardcoded'
        return None, 'not_found'
    
    def predict(self, pickup, dest):
        lane = f"{pickup} → {dest}"
        
        # Index prediction
        idx_pred = None
        if lane in self.lane_multipliers:
            idx_pred = self.current_index * self.lane_multipliers[lane]
        
        # Shrinkage prediction
        p_prior = self.pickup_priors.get(pickup, self.global_mean)
        d_prior = self.dest_priors.get(dest, self.global_mean)
        city_prior = (p_prior + d_prior) / 2
        
        if lane in self.lane_stats:
            stats = self.lane_stats[lane]
            lam = stats['lane_n'] / (stats['lane_n'] + self.k)
            shrink_pred = lam * stats['lane_mean'] + (1 - lam) * city_prior
            n_trips = stats['lane_n']
        else:
            shrink_pred = city_prior
            n_trips = 0
        
        # Combine
        if idx_pred is not None:
            final = (idx_pred + shrink_pred) / 2
            method = 'Index + Shrinkage'
        else:
            final = shrink_pred
            method = 'Shrinkage only'
        
        confidence = 'High' if n_trips >= 20 else ('Medium' if n_trips >= 5 else 'Low')
        
        return {
            'lane': lane, 'predicted_cpk': final, 'method': method,
            'confidence': confidence, 'n_trips': n_trips
        }

print("✅ RareLanePredictor class defined")

✅ RareLanePredictor class defined


In [115]:
# Test the predictor
print("=" * 70)
print("VALIDATION: Index + Shrinkage Model")
print("=" * 70)

predictor = RareLanePredictor(rare_lane_artifacts)

test_cases = [
    ('جدة', 'الرياض', 'High volume'),
    ('الجبيل', 'جدة', 'High volume'),
    ('ينبع', 'تبوك', 'Medium volume'),
    ('سدير', 'جازان', 'Sparse lane'),
]

for pickup, dest, desc in test_cases:
    result = predictor.predict(pickup, dest)
    dist, src = predictor.get_distance(pickup, dest)
    print(f"\n{desc}: {result['lane']}")
    print(f"   CPK: {result['predicted_cpk']:.3f} SAR/km")
    print(f"   Method: {result['method']} | Confidence: {result['confidence']} (n={result['n_trips']})")
    if dist:
        print(f"   Distance: {dist:.0f} km ({src}) | Total: {result['predicted_cpk']*dist:.0f} SAR")

VALIDATION: Index + Shrinkage Model

High volume: جدة → الرياض
   CPK: 1.556 SAR/km
   Method: Index + Shrinkage | Confidence: High (n=5227)
   Distance: 949 km (historical) | Total: 1477 SAR

High volume: الجبيل → جدة
   CPK: 1.775 SAR/km
   Method: Index + Shrinkage | Confidence: High (n=9939)
   Distance: 1412 km (historical) | Total: 2507 SAR

Medium volume: ينبع → تبوك
   CPK: 2.112 SAR/km
   Method: Index + Shrinkage | Confidence: High (n=87)
   Distance: 744 km (historical) | Total: 1571 SAR

Sparse lane: سدير → جازان
   CPK: 1.733 SAR/km
   Method: Index + Shrinkage | Confidence: Medium (n=8)
   Distance: 1077 km (historical) | Total: 1865 SAR


In [116]:
# Quantitative validation on holdout
print("\n" + "=" * 70)
print("QUANTITATIVE VALIDATION: Index + Shrinkage")
print("=" * 70)

# Use most recent 30 days as holdout
holdout = df[df['days_ago'] <= 30].copy()
train_for_val = df[df['days_ago'] > 30].copy()

print(f"\nHoldout: {len(holdout):,} trips (last 30 days)")
print(f"Training: {len(train_for_val):,} trips")

# Predict on holdout
predictions = []
actuals = []

for _, row in holdout.iterrows():
    result = predictor.predict(row['pickup_city'], row['destination_city'])
    predictions.append(result['predicted_cpk'])
    actuals.append(row['cost_per_km'])

predictions = np.array(predictions)
actuals = np.array(actuals)
residuals = predictions - actuals

print(f"\nIndex + Shrinkage Model Performance:")
print(f"   MAE:  {np.mean(np.abs(residuals)):.4f} SAR/km")
print(f"   RMSE: {np.sqrt(np.mean(residuals**2)):.4f} SAR/km")
print(f"   Bias: {np.mean(residuals):.4f} SAR/km")
print(f"   Within ±0.25: {(np.abs(residuals) <= 0.25).mean()*100:.1f}%")
print(f"   Within ±0.50: {(np.abs(residuals) <= 0.50).mean()*100:.1f}%")


QUANTITATIVE VALIDATION: Index + Shrinkage

Holdout: 714 trips (last 30 days)
Training: 84,234 trips

Index + Shrinkage Model Performance:
   MAE:  0.2054 SAR/km
   RMSE: 0.2655 SAR/km
   Bias: -0.1385 SAR/km
   Within ±0.25: 76.6%
   Within ±0.50: 91.3%


---
# 🔮 MODEL 3: PROVINCE BLEND 0.7 + LAD (new_lane_model_blend.pkl)

For lanes with **NO** historical data. Uses:
- **Province-level CPK** (13 official Saudi provinces from `region_test.csv` 'province' column)
- **City decomposition** (pickup_mult × dest_mult × index)
- **LAD (L1) calibration** to minimize MAE on predictions

**Key difference from Index+Shrinkage:**
- Province Blend uses **13 provinces** (e.g., "Riyadh Province", "Eastern Province", "Makkah Province")
- Index+Shrinkage fallback uses **5 regions** (Eastern, Western, Central, Northern, Southern)

**Formula:**
```
raw_cpk = 0.7 × Province_CPK + 0.3 × City_Decomposition
calibrated_cpk = LAD_slope × raw_cpk + LAD_intercept
```

In [129]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║               PROVINCE BLEND + LAD PARAMETERS                        ║
# ╚══════════════════════════════════════════════════════════════════════╝

PROVINCE_WEIGHT = 0.7       # Weight for province-level prediction (vs city decomposition)
LAD_ENABLED = True          # Enable LAD calibration (minimizes MAE)
HOLDOUT_DAYS = 30           # Days to hold out for LAD calibration fitting

print("Province Blend + LAD Parameters:")
print(f"   Province weight: {PROVINCE_WEIGHT} (city decomp: {1-PROVINCE_WEIGHT})")
print(f"   LAD calibration: {'Enabled' if LAD_ENABLED else 'Disabled'}")
print(f"   Holdout for fitting: {HOLDOUT_DAYS} days")

Province Blend + LAD Parameters:
   Province weight: 0.7 (city decomp: 0.30000000000000004)
   LAD calibration: Enabled
   Holdout for fitting: 30 days


In [130]:
# Build Province-level CPK matrix (13 official Saudi provinces)
print("Building Province Blend Model...")

# Get unique provinces (13 provinces from region_test.csv 'province' column)
unique_provinces = set(city_to_province.values()) - {None, '', np.nan}
print(f"   Unique provinces: {len(unique_provinces)}")
print(f"   Province list: {sorted(unique_provinces)}")

# Build province CPK matrix (uses 13 provinces for more granularity)
df_province = df[df['pickup_province'].notna() & df['dest_province'].notna()]
province_cpk = df_province.groupby(
    ['pickup_province', 'dest_province']
)['cost_per_km'].median().to_dict()

print(f"   Province corridors: {len(province_cpk)}")

# City decomposition multipliers (used for city-level component of blend)
pickup_city_mult = df_indexed.groupby('pickup_city')['multiplier'].median().to_dict()
dest_city_mult = df_indexed.groupby('destination_city')['multiplier'].median().to_dict()

print(f"   Pickup city multipliers: {len(pickup_city_mult)}")
print(f"   Dest city multipliers: {len(dest_city_mult)}")

Building Province Blend Model...
   Unique provinces: 13
   Province list: ['Al Bahah Province', 'Al Jouf Province', 'Asir Province', 'Eastern Province', 'Hail Province', 'Jazan Province', 'Madinah Province', 'Makkah Province', 'Najran Province', 'Northern Borders Province', 'Qassim Province', 'Riyadh Province', 'Tabuk Province']
   Province corridors: 77
   Pickup city multipliers: 35
   Dest city multipliers: 62


In [131]:
# Define raw blend prediction function (before LAD calibration)
def predict_blend_raw(pickup, dest, province_weight=PROVINCE_WEIGHT):
    """
    Predict raw CPK using Province Blend (before LAD calibration).
    
    Uses 13 Saudi provinces from region_test.csv 'province' column:
    - e.g., "Riyadh Province", "Eastern Province", "Makkah Province", etc.
    
    NOT the 5 macro-regions (Eastern, Western, Central, Northern, Southern)
    which are used by Index+Shrinkage fallback.
    """
    # Province prediction (13 provinces)
    p_province = city_to_province.get(pickup)
    d_province = city_to_province.get(dest)
    
    province_pred = None
    if p_province and d_province:
        province_pred = province_cpk.get((p_province, d_province))
    
    # City decomposition prediction
    city_pred = None
    p_mult = pickup_city_mult.get(pickup)
    d_mult = dest_city_mult.get(dest)
    if p_mult and d_mult:
        city_pred = current_index * p_mult * d_mult
    
    # Blend: 70% province + 30% city decomposition
    if province_pred is not None and city_pred is not None:
        return province_weight * province_pred + (1 - province_weight) * city_pred
    elif province_pred is not None:
        return province_pred
    elif city_pred is not None:
        return city_pred
    else:
        return global_mean  # Ultimate fallback

print("✅ predict_blend_raw function defined (uses 13 provinces)")

✅ predict_blend_raw function defined (uses 13 provinces)


In [132]:
# Fit LAD (L1) calibration
print("\nFitting LAD Calibration...")

# Generate predictions on training data
blend_preds_train = []
actuals_train = []

for _, row in df.iterrows():
    pred = predict_blend_raw(row['pickup_city'], row['destination_city'])
    if pred is not None:
        blend_preds_train.append(pred)
        actuals_train.append(row['cost_per_km'])

blend_preds_train = np.array(blend_preds_train)
actuals_train = np.array(actuals_train)

print(f"   Training samples: {len(blend_preds_train):,}")

# Fit LAD regression: actual = m * predicted + c
def lad_loss(params, x, y):
    m, c = params
    return np.sum(np.abs(y - (m * x + c)))

# OLS for initial guess
ols_coeffs = np.polyfit(blend_preds_train, actuals_train, 1)
initial_guess = [ols_coeffs[0], ols_coeffs[1]]

# LAD optimization
result = minimize(lad_loss, initial_guess, args=(blend_preds_train, actuals_train), method='Nelder-Mead')
lad_slope, lad_intercept = result.x

print(f"\n✅ LAD Calibration Fitted")
print(f"   Formula: P_calibrated = {lad_slope:.4f} × P_raw + {lad_intercept:.4f}")
print(f"   OLS comparison: P_calibrated = {ols_coeffs[0]:.4f} × P_raw + {ols_coeffs[1]:.4f}")


Fitting LAD Calibration...
   Training samples: 84,948

✅ LAD Calibration Fitted
   Formula: P_calibrated = 0.9800 × P_raw + -0.0108
   OLS comparison: P_calibrated = 0.8533 × P_raw + 0.2614


In [133]:
# Compare raw vs calibrated performance
print("\n" + "=" * 70)
print("LAD CALIBRATION IMPACT")
print("=" * 70)

raw_residuals = blend_preds_train - actuals_train
calibrated_preds = lad_slope * blend_preds_train + lad_intercept
calibrated_residuals = calibrated_preds - actuals_train

print(f"\n{'Metric':<20} {'Raw Blend':<15} {'LAD Calibrated':<15} {'Improvement':<15}")
print("-" * 65)

raw_mae = np.mean(np.abs(raw_residuals))
cal_mae = np.mean(np.abs(calibrated_residuals))
print(f"{'MAE':<20} {raw_mae:<15.4f} {cal_mae:<15.4f} {(raw_mae-cal_mae)/raw_mae*100:>+.1f}%")

raw_bias = np.mean(raw_residuals)
cal_bias = np.mean(calibrated_residuals)
print(f"{'Bias':<20} {raw_bias:<15.4f} {cal_bias:<15.4f} {abs(cal_bias)-abs(raw_bias):>+.4f}")

raw_rmse = np.sqrt(np.mean(raw_residuals**2))
cal_rmse = np.sqrt(np.mean(calibrated_residuals**2))
print(f"{'RMSE':<20} {raw_rmse:<15.4f} {cal_rmse:<15.4f} {(raw_rmse-cal_rmse)/raw_rmse*100:>+.1f}%")

raw_25 = (np.abs(raw_residuals) <= 0.25).mean() * 100
cal_25 = (np.abs(calibrated_residuals) <= 0.25).mean() * 100
print(f"{'Within ±0.25':<20} {raw_25:<15.1f}% {cal_25:<15.1f}% {cal_25-raw_25:>+.1f}pp")

raw_50 = (np.abs(raw_residuals) <= 0.50).mean() * 100
cal_50 = (np.abs(calibrated_residuals) <= 0.50).mean() * 100
print(f"{'Within ±0.50':<20} {raw_50:<15.1f}% {cal_50:<15.1f}% {cal_50-raw_50:>+.1f}pp")


LAD CALIBRATION IMPACT

Metric               Raw Blend       LAD Calibrated  Improvement    
-----------------------------------------------------------------
MAE                  0.3260          0.3226          +1.0%
Bias                 0.0140          -0.0344         +0.0204
RMSE                 0.4812          0.4808          +0.1%
Within ±0.25         51.6           % 52.5           % +1.0pp
Within ±0.50         84.7           % 84.9           % +0.3pp


In [134]:
# Package new_lane_model_blend.pkl
blend_artifacts = {
    'config': {
        'regional_weight': PROVINCE_WEIGHT,  # Keep name for backward compatibility with app
        'province_weight': PROVINCE_WEIGHT,
        'lad_enabled': LAD_ENABLED,
        'lad_slope': float(lad_slope),
        'lad_intercept': float(lad_intercept),
        'bias_correction': 0,  # Deprecated - LAD handles this now
        'training_date': str(df['date'].max()),
        'n_trips': len(df),
        'n_lanes': df['lane'].nunique(),
        'model_type': 'Province Blend 0.7 + LAD',
        'n_provinces': len(set(city_to_province.values())),  # Should be ~13
        'n_regions': len(set(city_to_region.values()))  # Should be 5
    },
    
    # Province-level (13 Saudi provinces) - for Blend model's primary grouping
    'province_cpk': province_cpk,  # (province, province) -> CPK
    'city_to_province': city_to_province,  # city -> province (13 provinces)
    
    # Region-level (5 regions) - for backward compatibility with app's regional fallback
    'regional_cpk': regional_cpk_5,  # (region, region) -> CPK
    'city_to_region': city_to_region_5,  # city -> region (5 regions)
    
    # City decomposition (30% weight in blend)
    'current_index': current_index,
    'benchmark_lanes': benchmark_lanes,
    'pickup_city_mult': pickup_city_mult,
    'dest_city_mult': dest_city_mult,
    
    # LAD calibration
    'lad_slope': float(lad_slope),
    'lad_intercept': float(lad_intercept),
}

with open(f'{EXPORT_DIR}/new_lane_model_blend.pkl', 'wb') as f:
    pickle.dump(blend_artifacts, f)

print(f"\n✅ Saved: {EXPORT_DIR}/new_lane_model_blend.pkl")
print(f"   Province corridors: {len(province_cpk)} (13 provinces)")
print(f"   Region corridors: {len(regional_cpk_5)} (5 regions)")
print(f"   City-to-province: {len(city_to_province)} mappings")
print(f"   City-to-region: {len(city_to_region_5)} mappings")


✅ Saved: final_models/new_lane_model_blend.pkl
   Province corridors: 77 (13 provinces)
   Region corridors: 24 (5 regions)
   City-to-province: 527 mappings
   City-to-region: 528 mappings


### 🧪 VALIDATION: Province Blend + LAD Model

In [135]:
# Define calibrated prediction function
def predict_blend_calibrated(pickup, dest, model=blend_artifacts):
    """
    Predict CPK using Province Blend + LAD calibration.
    """
    province_weight = model['config']['province_weight']
    lad_slope = model['lad_slope']
    lad_intercept = model['lad_intercept']
    
    # Province prediction
    p_province = model['city_to_province'].get(pickup)
    d_province = model['city_to_province'].get(dest)
    
    province_pred = None
    if p_province and d_province:
        province_pred = model['province_cpk'].get((p_province, d_province))
    
    # City decomposition
    city_pred = None
    p_mult = model['pickup_city_mult'].get(pickup)
    d_mult = model['dest_city_mult'].get(dest)
    if p_mult and d_mult:
        city_pred = model['current_index'] * p_mult * d_mult
    
    # Blend
    if province_pred is not None and city_pred is not None:
        raw_pred = province_weight * province_pred + (1 - province_weight) * city_pred
        method = f'Blend ({province_weight:.0%} Province)'
    elif province_pred is not None:
        raw_pred = province_pred
        method = 'Province only'
    elif city_pred is not None:
        raw_pred = city_pred
        method = 'City decomposition only'
    else:
        return None
    
    # Apply LAD calibration
    calibrated = lad_slope * raw_pred + lad_intercept
    
    return {
        'raw_cpk': raw_pred,
        'predicted_cpk': calibrated,
        'method': method + ' + LAD',
        'province_cpk': province_pred,
        'city_decomp_cpk': city_pred,
        'pickup_province': p_province,
        'dest_province': d_province
    }

print("✅ predict_blend_calibrated function defined")

✅ predict_blend_calibrated function defined


In [136]:
# Test predictions
print("=" * 70)
print("VALIDATION: Province Blend + LAD Model")
print("=" * 70)

test_lanes = [
    ('الرياض', 'جدة'),
    ('الدمام', 'الرياض'),
    ('جدة', 'المدينة المنورة'),
    ('تبوك', 'الدمام'),
    ('الجبيل', 'الرياض'),
]

for pickup, dest in test_lanes:
    result = predict_blend_calibrated(pickup, dest)
    if result:
        print(f"\n{pickup} → {dest}:")
        print(f"   Raw: {result['raw_cpk']:.3f} → Calibrated: {result['predicted_cpk']:.3f} SAR/km")
        print(f"   Method: {result['method']}")
        if result['province_cpk']:
            print(f"   Province ({result['pickup_province']}→{result['dest_province']}): {result['province_cpk']:.3f}")
        if result['city_decomp_cpk']:
            print(f"   City decomp: {result['city_decomp_cpk']:.3f}")
    else:
        print(f"\n{pickup} → {dest}: No prediction (missing data)")

VALIDATION: Province Blend + LAD Model

الرياض → جدة:
   Raw: 1.705 → Calibrated: 1.660 SAR/km
   Method: Blend (70% Province) + LAD
   Province (Riyadh Province→Makkah Province): 1.585
   City decomp: 1.985

الدمام → الرياض:
   Raw: 2.503 → Calibrated: 2.442 SAR/km
   Method: Blend (70% Province) + LAD
   Province (Eastern Province→Riyadh Province): 2.660
   City decomp: 2.135

جدة → المدينة المنورة:
   Raw: 2.106 → Calibrated: 2.053 SAR/km
   Method: Blend (70% Province) + LAD
   Province (Makkah Province→Madinah Province): 2.075
   City decomp: 2.180

تبوك → الدمام:
   Raw: 1.719 → Calibrated: 1.674 SAR/km
   Method: Blend (70% Province) + LAD
   Province (Tabuk Province→Eastern Province): 1.592
   City decomp: 2.015

الجبيل → الرياض:
   Raw: 2.437 → Calibrated: 2.378 SAR/km
   Method: Blend (70% Province) + LAD
   Province (Eastern Province→Riyadh Province): 2.660
   City decomp: 1.918


In [137]:
# Leave-one-out validation for NEW lane scenario
print("\n" + "=" * 70)
print("LEAVE-ONE-OUT VALIDATION: Simulating New Lanes")
print("=" * 70)

# For each lane with few trips, pretend it's "new" and predict
lane_trip_counts = df.groupby('lane').size()
sparse_lanes = lane_trip_counts[lane_trip_counts <= 5].index.tolist()[:50]  # Test first 50

loo_results = []
for lane in sparse_lanes:
    pickup, dest = lane.split(' → ')
    actual_cpk = df[df['lane'] == lane]['cost_per_km'].median()
    
    result = predict_blend_calibrated(pickup, dest)
    if result:
        loo_results.append({
            'lane': lane,
            'actual': actual_cpk,
            'predicted': result['predicted_cpk'],
            'residual': result['predicted_cpk'] - actual_cpk
        })

if loo_results:
    loo_df = pd.DataFrame(loo_results)
    print(f"\nTested {len(loo_df)} sparse lanes (≤5 trips):")
    print(f"   MAE:  {loo_df['residual'].abs().mean():.4f} SAR/km")
    print(f"   Bias: {loo_df['residual'].mean():.4f} SAR/km")
    print(f"   Within ±0.25: {(loo_df['residual'].abs() <= 0.25).mean()*100:.1f}%")
    print(f"   Within ±0.50: {(loo_df['residual'].abs() <= 0.50).mean()*100:.1f}%")


LEAVE-ONE-OUT VALIDATION: Simulating New Lanes

Tested 50 sparse lanes (≤5 trips):
   MAE:  0.5231 SAR/km
   Bias: -0.2366 SAR/km
   Within ±0.25: 40.0%
   Within ±0.50: 64.0%


---
# 💾 EXPORT REFERENCE DATA & CONFIG

In [84]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║                    REFERENCE DATA PARAMETERS                         ║
# ╚══════════════════════════════════════════════════════════════════════╝

RECENCY_CUTOFF_DAYS = 90    # Days to consider for recency model in app

print("Reference Data Parameters:")
print(f"   Recency cutoff: {RECENCY_CUTOFF_DAYS} days")

Reference Data Parameters:
   Recency cutoff: 90 days


In [85]:
# Create config.pkl
DISTANCE_LOOKUP = df[df['distance'] > 0].groupby('lane')['distance'].median().to_dict()

BACKHAUL_LOOKUP = {}
if 'destination_city' in backhaul_df.columns and 'backhaul_probability_pct' in backhaul_df.columns:
    BACKHAUL_LOOKUP = backhaul_df.set_index('destination_city')['backhaul_probability_pct'].to_dict()

config = {
    'FEATURES': ['entity_mapping', 'pickup_city', 'destination_city', 'commodity', 
                 'vehicle_type', 'distance', 'weight'],
    'ENTITY_MAPPING': ENTITY_MAPPING,
    'DISTANCE_LOOKUP': DISTANCE_LOOKUP,
    'BACKHAUL_LOOKUP': BACKHAUL_LOOKUP,
    'RECENCY_CUTOFF_DAYS': RECENCY_CUTOFF_DAYS,
}

with open(f'{EXPORT_DIR}/config.pkl', 'wb') as f:
    pickle.dump(config, f)

print(f"✅ Saved: {EXPORT_DIR}/config.pkl")
print(f"   Distance lookup: {len(DISTANCE_LOOKUP)} lanes")
print(f"   Backhaul lookup: {len(BACKHAUL_LOOKUP)} cities")

✅ Saved: final_models/config.pkl
   Distance lookup: 299 lanes
   Backhaul lookup: 24 cities


In [86]:
# Export reference_data.csv
export_cols = [
    'pickup_date', 'entity_mapping', 'commodity', 'vehicle_type',
    'pickup_city', 'destination_city', 'distance', 'weight',
    'total_carrier_price', 'days_ago', 'lane',
    'container', 'is_multistop', 'dest_backhaul_prob', 'lane_imbalance', 'lane_load_count'
]

for col in export_cols:
    if col not in df.columns:
        df[col] = 0

if 'total_shipper_price' in df.columns:
    export_cols.insert(export_cols.index('days_ago'), 'total_shipper_price')

df[export_cols].to_csv(f'{EXPORT_DIR}/reference_data.csv', index=False)

print(f"\n✅ Saved: {EXPORT_DIR}/reference_data.csv")
print(f"   Rows: {len(df):,}")
print(f"   Columns: {len(export_cols)}")


✅ Saved: final_models/reference_data.csv
   Rows: 84,948
   Columns: 17


In [87]:
# Export distance_matrix.pkl
with open(f'{EXPORT_DIR}/distance_matrix.pkl', 'wb') as f:
    pickle.dump(hardcoded_distances, f)

print(f"✅ Saved: {EXPORT_DIR}/distance_matrix.pkl")
print(f"   City pairs: {len(hardcoded_distances):,}")

✅ Saved: final_models/distance_matrix.pkl
   City pairs: 4,492


In [88]:
# Copy city normalization file
if os.path.exists(CITY_REGIONS_FILE):
    shutil.copy(CITY_REGIONS_FILE, f'{EXPORT_DIR}/city_normalization_with_regions.csv')
    print(f"✅ Copied: {EXPORT_DIR}/city_normalization_with_regions.csv")

---
# 🤖 MODEL 4: CATBOOST (Optional)

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║                     CATBOOST PARAMETERS                              ║
# ╚══════════════════════════════════════════════════════════════════════╝

TRAIN_CATBOOST = False       # Set True to train CatBoost model
CATBOOST_ITERATIONS = 300
CATBOOST_DEPTH = 9
CATBOOST_LR = 0.05
WEIGHT_HALF_LIFE_DAYS = 45   # Sample weighting decay

print("CatBoost Parameters:")
print(f"   Training: {'Enabled' if TRAIN_CATBOOST else 'Disabled'}")
if TRAIN_CATBOOST:
    print(f"   Iterations: {CATBOOST_ITERATIONS}")
    print(f"   Depth: {CATBOOST_DEPTH}")
    print(f"   Learning rate: {CATBOOST_LR}")
    print(f"   Weight half-life: {WEIGHT_HALF_LIFE_DAYS} days")

In [ ]:
if TRAIN_CATBOOST:
    from catboost import CatBoostRegressor, Pool
    from sklearn.metrics import mean_absolute_error, r2_score
    
    print("Training CatBoost model...")
    
    FEATURES = ['entity_mapping', 'pickup_city', 'destination_city', 
                'commodity', 'vehicle_type', 'distance', 'weight']
    CAT_FEATURES = ['entity_mapping', 'pickup_city', 'destination_city', 
                    'commodity', 'vehicle_type']
    CAT_INDICES = [FEATURES.index(f) for f in CAT_FEATURES]
    
    # Time-based split
    recent_df = df[df['days_ago'] <= 60].sort_values('days_ago')
    n_test = int(len(recent_df) * 0.2)
    
    test_df = recent_df.head(n_test)
    train_df = recent_df.iloc[n_test:].copy()
    
    train_df['sample_weight'] = np.exp(-np.log(2) * train_df['days_ago'] / WEIGHT_HALF_LIFE_DAYS)
    
    print(f"   Training: {len(train_df):,} samples")
    print(f"   Testing: {len(test_df):,} samples")
    
    carrier_model = CatBoostRegressor(
        iterations=CATBOOST_ITERATIONS,
        depth=CATBOOST_DEPTH,
        learning_rate=CATBOOST_LR,
        random_seed=42,
        verbose=0
    )
    
    carrier_pool = Pool(
        train_df[FEATURES],
        train_df['total_carrier_price'],
        cat_features=CAT_INDICES,
        weight=train_df['sample_weight']
    )
    carrier_model.fit(carrier_pool)
    
    # Evaluate
    test_pred = carrier_model.predict(test_df[FEATURES])
    mae = mean_absolute_error(test_df['total_carrier_price'], test_pred)
    r2 = r2_score(test_df['total_carrier_price'], test_pred)
    errors = np.abs(test_pred - test_df['total_carrier_price']) / test_df['total_carrier_price']
    
    print(f"\n✅ CatBoost Model Performance:")
    print(f"   MAE: {mae:,.0f} SAR")
    print(f"   R²: {r2:.3f}")
    print(f"   Within 10%: {(errors <= 0.10).mean()*100:.1f}%")
    print(f"   Within 15%: {(errors <= 0.15).mean()*100:.1f}%")
    print(f"   Within 20%: {(errors <= 0.20).mean()*100:.1f}%")
    
    # Save
    carrier_model.save_model(f'{EXPORT_DIR}/carrier_model.json', format='json')
    print(f"\n✅ Saved: {EXPORT_DIR}/carrier_model.json")
    
    config['FEATURES'] = FEATURES
    config['CAT_FEATURES'] = CAT_FEATURES
    config['CAT_INDICES'] = CAT_INDICES
    
    with open(f'{EXPORT_DIR}/config.pkl', 'wb') as f:
        pickle.dump(config, f)
else:
    print("⏭️ Skipping CatBoost training (TRAIN_CATBOOST = False)")

---
# 📋 EXPORT SUMMARY

In [ ]:
print("=" * 70)
print("📦 EXPORT COMPLETE")
print("=" * 70)

print(f"\n📁 Output directory: {os.path.abspath(EXPORT_DIR)}")
print("\nFiles created:")
for f in sorted(os.listdir(EXPORT_DIR)):
    if f.startswith('.'): continue
    size = os.path.getsize(f'{EXPORT_DIR}/{f}')
    if size > 1024*1024:
        size_str = f"{size/1024/1024:.1f} MB"
    elif size > 1024:
        size_str = f"{size/1024:.1f} KB"
    else:
        size_str = f"{size} B"
    print(f"   {f}: {size_str}")

print(f"\n📊 Training Data Summary:")
print(f"   Trips: {len(df):,}")
print(f"   Lanes: {df['lane'].nunique()}")
print(f"   Date range: {df['date'].min()} to {df['date'].max()}")
print(f"   Entity: {ENTITY_MAPPING}")
print(f"   Vehicles: {VEHICLE_TYPES}")

print(f"\n🎯 Model Components:")
print(f"   Market index: {current_index:.4f} SAR/km")
print(f"   Lane multipliers: {len(lane_multipliers)}")
print(f"   Province corridors: {len(province_cpk)}")
print(f"   5-Region corridors: {len(regional_cpk_5)}")
print(f"   City-province mappings: {len(city_to_province)}")
print(f"   Distance pairs: {len(historical_distances)} historical + {len(hardcoded_distances)} hardcoded")

print(f"\n📐 LAD Calibration:")
print(f"   Formula: P_calibrated = {lad_slope:.4f} × P_raw + {lad_intercept:.4f}")
print(f"   MAE improvement: {(raw_mae-cal_mae)/raw_mae*100:.1f}%")

In [ ]:
# Final verification
print("\n" + "=" * 70)
print("🧪 FINAL VERIFICATION - Loading exported models")
print("=" * 70)

with open(f'{EXPORT_DIR}/rare_lane_models.pkl', 'rb') as f:
    loaded_rare = pickle.load(f)
print(f"\n✅ rare_lane_models.pkl loaded")
print(f"   Lane stats: {len(loaded_rare['lane_stats'])}")
print(f"   Current index: {loaded_rare['current_index']:.4f}")

with open(f'{EXPORT_DIR}/new_lane_model_blend.pkl', 'rb') as f:
    loaded_blend = pickle.load(f)
print(f"\n✅ new_lane_model_blend.pkl loaded")
print(f"   Model type: {loaded_blend['config'].get('model_type', 'Blend')}")
print(f"   Province weight: {loaded_blend['config']['province_weight']}")
print(f"   LAD slope: {loaded_blend['lad_slope']:.4f}")
print(f"   LAD intercept: {loaded_blend['lad_intercept']:.4f}")

with open(f'{EXPORT_DIR}/config.pkl', 'rb') as f:
    loaded_config = pickle.load(f)
print(f"\n✅ config.pkl loaded")
print(f"   Distance lookup: {len(loaded_config['DISTANCE_LOOKUP'])} lanes")

loaded_ref = pd.read_csv(f'{EXPORT_DIR}/reference_data.csv')
print(f"\n✅ reference_data.csv loaded")
print(f"   Rows: {len(loaded_ref):,}")

with open(f'{EXPORT_DIR}/distance_matrix.pkl', 'rb') as f:
    loaded_dist = pickle.load(f)
print(f"\n✅ distance_matrix.pkl loaded")
print(f"   City pairs: {len(loaded_dist):,}")

print("\n" + "=" * 70)
print("✅ ALL MODELS VERIFIED SUCCESSFULLY")
print("=" * 70)

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    DEPLOYMENT INSTRUCTIONS                          ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  1. Upload contents of final_models/ to GitHub:                      ║
║     → pricing_app/model_export/                                      ║
║                                                                      ║
║  2. Files to upload:                                                 ║
║     - config.pkl                                                     ║
║     - reference_data.csv                                             ║
║     - rare_lane_models.pkl                                           ║
║     - new_lane_model_blend.pkl (Province Blend 0.7 + LAD)            ║
║     - distance_matrix.pkl                                            ║
║     - city_normalization_with_regions.csv                            ║
║     - carrier_model.json (if CatBoost trained)                       ║
║                                                                      ║
║  3. Streamlit Cloud will auto-deploy on push                         ║
║                                                                      ║
║  4. Pricing Cascade (in app):                                        ║
║     • Priority 1: Recency (median of last 90 days)                   ║
║     • Priority 2: Index + Shrinkage (lanes WITH history)             ║
║     • Priority 3: Province Blend 0.7 + LAD (NEW lanes)               ║
║                                                                      ║
║  5. Province Blend + LAD Calibration:                                ║
║     raw = 0.7 × Province_CPK + 0.3 × City_Decomposition              ║
║     calibrated = {:.4f} × raw + {:.4f}                               ║
║                                                                      ║
║  6. Buy Price rounded to nearest 100 SAR                             ║
║     Sell Price = Buy × (1 + margin) rounded to nearest 50 SAR        ║
║     Margin based on backhaul probability (12-25%)                    ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""".format(lad_slope, lad_intercept))

In [146]:
city_regions_df.groupby(['province','region']).canonical.nunique()

province                   region  
Al Bahah Province          Southern     8
Al Jouf Province           Northern     4
Asir Province              Southern    21
Eastern Province           Central      2
                           Eastern     25
Hail Province              Northern     6
Jazan Province             Southern    11
Madinah Province           Northern     1
                           Western      9
Makkah Province            Western     30
Najran Province            Southern     6
Northern Borders Province  Northern     3
Qassim Province            Central      2
                           Northern    15
Riyadh Province            Central     24
                           Eastern      1
Tabuk Province             Northern     5
                           Western      3
Name: canonical, dtype: int64

In [147]:
# 1. Count unique cities per province/region pair
region_counts = city_regions_df.groupby(['province', 'region'])['canonical'].nunique().reset_index(name='city_count')

# 2. Sort by count (descending) so the most popular region is first
region_counts = region_counts.sort_values('city_count', ascending=False)

# 3. Drop duplicates on 'province' (keeping the first, which is the highest count)
# This creates a 1-to-1 mapping of Province -> Dominant Region
dominant_regions = region_counts.drop_duplicates('province', keep='first')

# 4. Convert to a dictionary for easy mapping
province_to_region_map = dict(zip(dominant_regions['province'], dominant_regions['region']))

# View the result
print(province_to_region_map)

{'Makkah Province': 'Western', 'Eastern Province': 'Eastern', 'Riyadh Province': 'Central', 'Asir Province': 'Southern', 'Qassim Province': 'Northern', 'Jazan Province': 'Southern', 'Madinah Province': 'Western', 'Al Bahah Province': 'Southern', 'Najran Province': 'Southern', 'Hail Province': 'Northern', 'Tabuk Province': 'Northern', 'Al Jouf Province': 'Northern', 'Northern Borders Province': 'Northern'}
